In [1]:
import sys
sys.path.append('..')
from src.data.docx  import read_docx,process_docx,find_content
import glob
import json
import os
import pandas as pd
import ast
import warnings
warnings.filterwarnings("ignore")
from src.models.pipeline import pipeline
from src.features.segment_transcript import CreateBlocks
from functools import reduce
import heapq
from gensim.corpora import Dictionary
from gensim import models
from gensim import similarities
import re
import statistics
import numpy as np
import seaborn as sns; sns.set()
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from IPython.display import display, HTML
from scipy.stats import entropy

In [3]:
df_google_form = pd.read_csv('../data/raw/forms/form-answers-poc.csv')
mycolumns = [
             '00:00-04:06','04:06-06:55','06:55-08:54',
             '08:54-10:39','10:39-13:03','13:03-14:56',
             '14:56-17:42','17:42-22:09','22:09-24:41',
             '24:41-27:11','27:11-30:18','30:18-33:39',
             '33:39-38:24','38:24-41:42','41:42-45:20',
             '45:20-49:47','49:47-52:52','52:52-53:30',
             'rating_00:00-04:06','rating_04:06-06:55','rating_06:55-08:54',
             'rating_08:54-10:39','rating_10:39-13:03','rating_13:03-14:56',
             'rating_14:56-17:42','rating_17:42-22:09','rating_22:09-24:41',
             'rating_24:41-27:11','rating_27:11-30:18','rating_30:18-33:39',
             'rating_33:39-38:24','rating_38:24-41:42','rating_41:42-45:20',
             'rating_45:20-49:47','rating_49:47-52:52','rating_52:52-53:30'
            ]
df_google_form = df_google_form.drop(columns=['מבין הפרקים הבאים, איזה אחד הכי מתאים לטווח הזמנים: 00:00-04:06 בסרטון'
                                              ,'חותמת זמן','Unnamed: 20'])
df_google_form.columns = mycolumns
df_results = pd.DataFrame()
questions = list(filter(lambda x: "rating" not in x,df_google_form.columns))
rating_confidence = list(filter(lambda x: "rating" in x,df_google_form.columns))
chapters_most_frequent = [df_google_form[col].mode().values.tolist() for col in questions]
ratings_avg = [df_google_form[col].mean() for col in rating_confidence]
ratings_var = [df_google_form[col].var() for col in rating_confidence]
df_results['most corr labeled chapters'] = chapters_most_frequent
df_results['rating mean confidence '] = ratings_avg
df_results['rating var confidence '] = ratings_var
df_results['labeled num choices'] = [len(df_google_form[col].value_counts()) for col in questions]
df_results['max entropy'] = [np.log2(len(df_google_form[col].value_counts())) for col in questions]
df_results['labeled choose entropy'] = [entropy(df_google_form[col].value_counts(normalize=True),base=2) for col in questions]
df_results['founded chapter corr'] = [max(topic_correlations) for topic_correlations in correlation]
dom_subsec_titles_ = [re.sub("\t",' ',tl) for tl in dom_subsec_titles]
df_results['labeled chapter corr'] = [[correlation[index][dom_subsec_titles_.index(ch)] for ch in ch_corr_labeled] 
                                      for index,ch_corr_labeled in enumerate(chapters_most_frequent)]
df_results

NameError: name 'correlation' is not defined